# Sparse Representation of Photometric Redshift PDFs

#### [MNRAS, 441, 3550](http://adsabs.harvard.edu/cgi-bin/bib_query?arXiv:1404.6442)

Sparse Representation of photometric redshift PDFs  is a novel technique to represent and store these PDFs in an efficient way without losing resolution or information. It works by creating a over determined dictionary of basis with Gaussian and Voigt profiles which cover the redshift space for several values for the center and width for these distributions. Usually we create less than 2^16 bases so we can represent them using 16 bits integers. By using an Orthogonal Matching Pursuit algorithm we look to minimize the number of bases used, without repetition, and the coefficients for each base can be also represented using another 16 bits integer. In this way, we can represent a PDF using only 32-bits integers per base when only 10-20 bases is necessary.


![32bit](images/32bit.png)
  <p style="text-align: center; color: blue;">A single four-byte integer scheme to store a single basis function in the sparse representation method.</p>

![bases](images/bases.png)
  <p style="text-align: center; color: blue;">A illustration of the bases used in the dictionary, each shaded region is composed by several functions of different widths, those colored shaded regions are full of bases. In a real dictionary the spacing between these functions is much tighter (~200 points in the redshift range)</p>
  


A Voigt profile is defined as:

$$ V(x;\sigma,\gamma) = \frac{1}{\sigma\sqrt{2\pi}} \operatorname{Re}\left[e^{-z^2} \left(1-\erf (-iz)\right)\right]
$$
![voigt](images/voigt.png)
  <p style="text-align: center; color: blue;">Different normalize $||\mathbf{d}||_2$ Voigt profile basis functions with the same mean, $\mu = 0.3$, and sigma, $ \sigma= 0.01$, for differentvalues of $\gamma$, which ranges from 0 (blue) to 1$\sigma$(red). Note that for $\gamma = 0$, we recover the standard Gaussian distribution. In a full dic-tionary, we create these profiles over the entire redshift range of thegalaxy sample for different values of $\sigma$.</p>

### Sparse representation, the problem

We want to represent each galaxy PDF $\mathbf{pz_k}$ as:

$$\mathbf{pz}_k = \mathbf{D} \boldsymbol{\delta}_k + \boldsymbol{\epsilon}_k$$

where  $\mathbf{D}$  is a dictionary or basis matrix of dimension $n \times m$, where $m > n$.  Thus, we have an over-determined problem as the number of basis functions, $m$, is much larger than the dimension, $n$,  of each PDF. We want to find, for each galaxy $k$, the optimal vector solution $\boldsymbol{\delta}_k$, which is determined such that its pseudo-norm $||\boldsymbol{\delta}_k||_0$ is minimized. Alternatively, this can be equivalently stated that we want to minimize the number of non-zero entries in the vector, $\boldsymbol{\delta}_k$, given the residual error $\boldsymbol{\epsilon}_k$.

We use an Orthogonal Matching Pursuit (OMP) approach for this, the steps are as follows:

- Initialize all variables. First, define the residual vector to be the original photo-$z$ PDF, $\boldsymbol{\epsilon}_k^{0} = \mathbf{pz}_k$. Second, create an empty set of cumulative selected basis functions, $\mathbf{B}_k$. Finally, set $\boldsymbol{\delta}_k = 0$, and define $i=0$ as the number of the current iteration.

- Compute the current set of basis functions. First, find the column vector, $\mathbf{d}_b$, from the dictionary matrix, $\mathbf{D}$, where $b$ is the index position that maximizes the projection of $\boldsymbol{\epsilon}_k^i$:
\begin{equation}
\mathbf{d}_b^i = \max_{\mathbf{d}_j \in \mathbf{D}} | \mathbf{d}_j^T \cdot \boldsymbol{\epsilon}_k^{i} | 
\end{equation}
Second, add this selected basis function to the set $\mathbf{B}_k$, i.e. $\mathbf{B}_k = (\mathbf{B}_k,\mathbf{d}_b^i)$.

- Orthogonally project the original \pz PDF onto the linear space spanned by the columns of all previously selected basis functions:
\begin{equation}
\mathbf{w}_k^i = \mathbf{B}_k^T \cdot \mathbf{pz}_k 
\end{equation}
where $\mathbf{w}_k^i$ is a temporary vector corresponding to the coefficients of the currently used basis functions in $\mathbf{B}_k$.

-  Complete the projection by updating the residuals by using the temporary vector $\mathbf{w}_k^i$:
\begin{equation}
\boldsymbol{\epsilon}_k^{i+1} = \mathbf{pz}_k - \mathbf{B}_k \cdot \mathbf{w}_k^i
\end{equation}

- Check the stopping criteria: $||\boldsymbol{\epsilon}_k^{i+1} ||_2 < \epsilon_{th}$, where $\epsilon_{th}$ is the threshold residual or  $i > i_{lim}$, where $i_{lim}$ is the number of required basis functions. If the pre-selected stopping criteria is met, the calculations are completed: $\boldsymbol{\delta}_k = \mathbf{w}_k^i$ and $\mathbf{pz}_k = \mathbf{D} \cdot \boldsymbo
l{\delta}_k + \boldsymbol{\epsilon}_k^{i+1}$, where $\boldsymbol{\delta}_k$ is sparse. Finally, the photo-$z$ PDF representation is defined:
\begin{equation}
\mathbf{pz}_k \approx \mathbf{D} \cdot \boldsymbol{\delta}_k
\end{equation}
On the other hand, if the predefined stopping criteria is not met, the iteration step is increased, $i = i + 1$, and steps $2$--$5$ are repeated by using the current residual vector. This process is repeated over all galaxies $k$, where $k=1,2,\dots,N$.


### Demo
Link to [demo](Sparse_demo.ipynb)